In [32]:
#Install pyton modules
#pip intall gym==0.26.2
#pip intall numpy

In [33]:
import gym
import numpy as np
import time

In [34]:
slipperyState = True

In [35]:
#Before Training

In [36]:
time.sleep(5)

In [37]:
# Set up a new environment
env=gym.make("FrozenLake-v1", is_slippery=slipperyState)
env.reset()
state = env.reset()[0]

#Check action size and atate size
action_size = env.action_space.n
state_size = env.observation_space.n   
print('action size=', action_size, 'state size=', state_size)

# Start with very small values for all our Q(s,a)
q_table = np.zeros([state_size, action_size])
print('Q Table Init = \n', q_table, '\nQ Table Shape = ', q_table.shape)

action size= 4 state size= 16
Q Table Init = 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 
Q Table Shape =  (16, 4)


In [38]:
#Setup and run enviroment to see what happens if no training
env=gym.make("FrozenLake-v1", render_mode='human', is_slippery=slipperyState)
env.reset()
state = env.reset()[0]
for steps in range(100):
    env.render()
    action = np.argmax(q_table[state, :])
    state, reward, done, done1, info = env.step(action)
    time.sleep(1)
    if done:
        break
    else:
        time.sleep(10)
        break
env.close()

In [49]:
#During Training (Not Shown)

In [50]:
time.sleep(5)

In [51]:
# Set up a new environment
env=gym.make("FrozenLake-v1", is_slippery=slipperyState)
env.reset()
state = env.reset()[0]

#Check action size and atate size
action_size = env.action_space.n
state_size = env.observation_space.n   
print('action size=', action_size, 'state size=', state_size)

# Start with very small values for all our Q(s,a)
q_table = np.zeros([state_size, action_size])
print('Q Table Init = \n', q_table, '\nQ Table Shape = ', q_table.shape)

action size= 4 state size= 16
Q Table Init = 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 
Q Table Shape =  (16, 4)


In [42]:
# It is common to leave Hyperparameters in ALL CAPS to easily locate them

EPOCHS=20000  # number of epochs/episodes to train for
ALPHA = 0.8 # aka the learning rate
GAMMA = 0.95 # aka the discount rate
# MAX_EPISODES = 100  # optional, also defined in env setup above

# Exploration vs. Exploitation parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.001             # Exponential decay rate for exploration prob

def epsilon_greedy_action_selection(epsilon, q_table, discrete_state):
    '''
    Returns an action for the agent. Note how it uses a random number to decide on
    exploration versus explotation trade-off.
    '''
    random_number = np.random.random()
    
    # EXPLOITATION, USE BEST Q(s,a) Value
    if random_number > epsilon:
        # Action row for a particular state
        state_row = q_table[discrete_state,:]
        # Index of highest action for state
        # Recall action is mapped to index (e.g. 0=LEFT, 1=DOWN, etc..)
        action = np.argmax(state_row)
    
    # EXPLORATION, USE A RANDOM ACTION
    else:
        # Return a random 0,1,2,3 action
        action = env.action_space.sample()
        
    return action

def compute_next_q_value(old_q_value, reward, next_optimal_q_value):
    
    return old_q_value +  ALPHA * (reward + GAMMA * next_optimal_q_value - old_q_value)

def reduce_epsilon(epsilon,epoch):
    
    return min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*epoch)

In [43]:
# Reset just in case, watch lecture on this.
q_table = np.zeros([state_size, action_size])
total_reward = 0
epsilon = 1

# List of rewards
rewards = []

In [44]:
#env=gym.make("FrozenLake-v1", render_mode='human', is_slippery=False)
#env.reset()
#state = env.reset()[0]

# Play 20k games
for episode in range(EPOCHS):
    print('Episode: ',episode)
    # Reset the environment
    state = env.reset()[0]
    done = False
    done1 = False
    total_rewards = 0
    
    while not done:
        action = epsilon_greedy_action_selection(epsilon,q_table, state)

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, done1, info = env.step(action)

        
        # Look up current/old qtable value Q(s_t,a_t)
        # print('state= ', state, 'action= ', action)
        old_q_value =  q_table[state,action]  

        # Get the next optimal Q-Value
        next_optimal_q_value = np.max(q_table[new_state, :])  

        # Compute next q value
        next_q = compute_next_q_value(old_q_value, reward, next_optimal_q_value)   

        # Update Q Table
        q_table[state,action] = next_q

        
        
        total_rewards = total_rewards + reward
        
        # Our new state is state
        state = new_state
                
    episode += 1
    # Reduce epsilon (because we need less and less exploration)
    epsilon = reduce_epsilon(epsilon,episode) 
    rewards.append(total_rewards)
    if episode % 1000 == 0:
        print('Episode: {} Reward: {} '.format(episode,np.sum(rewards))) 
            

#env.close()

Episode:  0
Episode:  1
Episode:  2
Episode:  3
Episode:  4
Episode:  5
Episode:  6
Episode:  7
Episode:  8
Episode:  9
Episode:  10
Episode:  11
Episode:  12
Episode:  13
Episode:  14
Episode:  15
Episode:  16
Episode:  17
Episode:  18
Episode:  19
Episode:  20
Episode:  21
Episode:  22
Episode:  23
Episode:  24
Episode:  25
Episode:  26
Episode:  27
Episode:  28
Episode:  29
Episode:  30
Episode:  31
Episode:  32
Episode:  33
Episode:  34
Episode:  35
Episode:  36
Episode:  37
Episode:  38
Episode:  39
Episode:  40
Episode:  41
Episode:  42
Episode:  43
Episode:  44
Episode:  45
Episode:  46
Episode:  47
Episode:  48
Episode:  49
Episode:  50
Episode:  51
Episode:  52
Episode:  53
Episode:  54
Episode:  55
Episode:  56
Episode:  57
Episode:  58
Episode:  59
Episode:  60
Episode:  61
Episode:  62
Episode:  63
Episode:  64
Episode:  65
Episode:  66
Episode:  67
Episode:  68
Episode:  69
Episode:  70
Episode:  71
Episode:  72
Episode:  73
Episode:  74
Episode:  75
Episode:  76
Episode: 

Episode:  3497
Episode:  3498
Episode:  3499
Episode:  3500
Episode:  3501
Episode:  3502
Episode:  3503
Episode:  3504
Episode:  3505
Episode:  3506
Episode:  3507
Episode:  3508
Episode:  3509
Episode:  3510
Episode:  3511
Episode:  3512
Episode:  3513
Episode:  3514
Episode:  3515
Episode:  3516
Episode:  3517
Episode:  3518
Episode:  3519
Episode:  3520
Episode:  3521
Episode:  3522
Episode:  3523
Episode:  3524
Episode:  3525
Episode:  3526
Episode:  3527
Episode:  3528
Episode:  3529
Episode:  3530
Episode:  3531
Episode:  3532
Episode:  3533
Episode:  3534
Episode:  3535
Episode:  3536
Episode:  3537
Episode:  3538
Episode:  3539
Episode:  3540
Episode:  3541
Episode:  3542
Episode:  3543
Episode:  3544
Episode:  3545
Episode:  3546
Episode:  3547
Episode:  3548
Episode:  3549
Episode:  3550
Episode:  3551
Episode:  3552
Episode:  3553
Episode:  3554
Episode:  3555
Episode:  3556
Episode:  3557
Episode:  3558
Episode:  3559
Episode:  3560
Episode:  3561
Episode:  3562
Episode:  

Episode:  6647
Episode:  6648
Episode:  6649
Episode:  6650
Episode:  6651
Episode:  6652
Episode:  6653
Episode:  6654
Episode:  6655
Episode:  6656
Episode:  6657
Episode:  6658
Episode:  6659
Episode:  6660
Episode:  6661
Episode:  6662
Episode:  6663
Episode:  6664
Episode:  6665
Episode:  6666
Episode:  6667
Episode:  6668
Episode:  6669
Episode:  6670
Episode:  6671
Episode:  6672
Episode:  6673
Episode:  6674
Episode:  6675
Episode:  6676
Episode:  6677
Episode:  6678
Episode:  6679
Episode:  6680
Episode:  6681
Episode:  6682
Episode:  6683
Episode:  6684
Episode:  6685
Episode:  6686
Episode:  6687
Episode:  6688
Episode:  6689
Episode:  6690
Episode:  6691
Episode:  6692
Episode:  6693
Episode:  6694
Episode:  6695
Episode:  6696
Episode:  6697
Episode:  6698
Episode:  6699
Episode:  6700
Episode:  6701
Episode:  6702
Episode:  6703
Episode:  6704
Episode:  6705
Episode:  6706
Episode:  6707
Episode:  6708
Episode:  6709
Episode:  6710
Episode:  6711
Episode:  6712
Episode:  

Episode:  8508
Episode:  8509
Episode:  8510
Episode:  8511
Episode:  8512
Episode:  8513
Episode:  8514
Episode:  8515
Episode:  8516
Episode:  8517
Episode:  8518
Episode:  8519
Episode:  8520
Episode:  8521
Episode:  8522
Episode:  8523
Episode:  8524
Episode:  8525
Episode:  8526
Episode:  8527
Episode:  8528
Episode:  8529
Episode:  8530
Episode:  8531
Episode:  8532
Episode:  8533
Episode:  8534
Episode:  8535
Episode:  8536
Episode:  8537
Episode:  8538
Episode:  8539
Episode:  8540
Episode:  8541
Episode:  8542
Episode:  8543
Episode:  8544
Episode:  8545
Episode:  8546
Episode:  8547
Episode:  8548
Episode:  8549
Episode:  8550
Episode:  8551
Episode:  8552
Episode:  8553
Episode:  8554
Episode:  8555
Episode:  8556
Episode:  8557
Episode:  8558
Episode:  8559
Episode:  8560
Episode:  8561
Episode:  8562
Episode:  8563
Episode:  8564
Episode:  8565
Episode:  8566
Episode:  8567
Episode:  8568
Episode:  8569
Episode:  8570
Episode:  8571
Episode:  8572
Episode:  8573
Episode:  

Episode:  9846
Episode:  9847
Episode:  9848
Episode:  9849
Episode:  9850
Episode:  9851
Episode:  9852
Episode:  9853
Episode:  9854
Episode:  9855
Episode:  9856
Episode:  9857
Episode:  9858
Episode:  9859
Episode:  9860
Episode:  9861
Episode:  9862
Episode:  9863
Episode:  9864
Episode:  9865
Episode:  9866
Episode:  9867
Episode:  9868
Episode:  9869
Episode:  9870
Episode:  9871
Episode:  9872
Episode:  9873
Episode:  9874
Episode:  9875
Episode:  9876
Episode:  9877
Episode:  9878
Episode:  9879
Episode:  9880
Episode:  9881
Episode:  9882
Episode:  9883
Episode:  9884
Episode:  9885
Episode:  9886
Episode:  9887
Episode:  9888
Episode:  9889
Episode:  9890
Episode:  9891
Episode:  9892
Episode:  9893
Episode:  9894
Episode:  9895
Episode:  9896
Episode:  9897
Episode:  9898
Episode:  9899
Episode:  9900
Episode:  9901
Episode:  9902
Episode:  9903
Episode:  9904
Episode:  9905
Episode:  9906
Episode:  9907
Episode:  9908
Episode:  9909
Episode:  9910
Episode:  9911
Episode:  

...

Episode: 12000 Reward: 4608.0 
Episode:  12000
Episode:  12001
Episode:  12002
Episode:  12003
Episode:  12004
Episode:  12005
Episode:  12006
Episode:  12007
Episode:  12008
Episode:  12009
Episode:  12010
Episode:  12011
Episode:  12012
Episode:  12013
Episode:  12014
Episode:  12015
Episode:  12016
Episode:  12017
Episode:  12018
Episode:  12019
Episode:  12020
Episode:  12021
Episode:  12022
Episode:  12023
Episode:  12024
Episode:  12025
Episode:  12026
Episode:  12027
Episode:  12028
Episode:  12029
Episode:  12030
Episode:  12031
Episode:  12032
Episode:  12033
Episode:  12034
Episode:  12035
Episode:  12036
Episode:  12037
Episode:  12038
Episode:  12039
Episode:  12040
Episode:  12041
Episode:  12042
Episode:  12043
Episode:  12044
Episode:  12045
Episode:  12046
Episode:  12047
Episode:  12048
Episode:  12049
Episode:  12050
Episode:  12051
Episode:  12052
Episode:  12053
Episode:  12054
Episode:  12055
Episode:  12056
Episode:  12057
Episode:  12058
Episode:  12059
Episode: 

Episode:  13627
Episode:  13628
Episode:  13629
Episode:  13630
Episode:  13631
Episode:  13632
Episode:  13633
Episode:  13634
Episode:  13635
Episode:  13636
Episode:  13637
Episode:  13638
Episode:  13639
Episode:  13640
Episode:  13641
Episode:  13642
Episode:  13643
Episode:  13644
Episode:  13645
Episode:  13646
Episode:  13647
Episode:  13648
Episode:  13649
Episode:  13650
Episode:  13651
Episode:  13652
Episode:  13653
Episode:  13654
Episode:  13655
Episode:  13656
Episode:  13657
Episode:  13658
Episode:  13659
Episode:  13660
Episode:  13661
Episode:  13662
Episode:  13663
Episode:  13664
Episode:  13665
Episode:  13666
Episode:  13667
Episode:  13668
Episode:  13669
Episode:  13670
Episode:  13671
Episode:  13672
Episode:  13673
Episode:  13674
Episode:  13675
Episode:  13676
Episode:  13677
Episode:  13678
Episode:  13679
Episode:  13680
Episode:  13681
Episode:  13682
Episode:  13683
Episode:  13684
Episode:  13685
Episode:  13686
Episode:  13687
Episode:  13688
Episode:

Episode:  14897
Episode:  14898
Episode:  14899
Episode:  14900
Episode:  14901
Episode:  14902
Episode:  14903
Episode:  14904
Episode:  14905
Episode:  14906
Episode:  14907
Episode:  14908
Episode:  14909
Episode:  14910
Episode:  14911
Episode:  14912
Episode:  14913
Episode:  14914
Episode:  14915
Episode:  14916
Episode:  14917
Episode:  14918
Episode:  14919
Episode:  14920
Episode:  14921
Episode:  14922
Episode:  14923
Episode:  14924
Episode:  14925
Episode:  14926
Episode:  14927
Episode:  14928
Episode:  14929
Episode:  14930
Episode:  14931
Episode:  14932
Episode:  14933
Episode:  14934
Episode:  14935
Episode:  14936
Episode:  14937
Episode:  14938
Episode:  14939
Episode:  14940
Episode:  14941
Episode:  14942
Episode:  14943
Episode:  14944
Episode:  14945
Episode:  14946
Episode:  14947
Episode:  14948
Episode:  14949
Episode:  14950
Episode:  14951
Episode:  14952
Episode:  14953
Episode:  14954
Episode:  14955
Episode:  14956
Episode:  14957
Episode:  14958
Episode:

Episode:  17019
Episode:  17020
Episode:  17021
Episode:  17022
Episode:  17023
Episode:  17024
Episode:  17025
Episode:  17026
Episode:  17027
Episode:  17028
Episode:  17029
Episode:  17030
Episode:  17031
Episode:  17032
Episode:  17033
Episode:  17034
Episode:  17035
Episode:  17036
Episode:  17037
Episode:  17038
Episode:  17039
Episode:  17040
Episode:  17041
Episode:  17042
Episode:  17043
Episode:  17044
Episode:  17045
Episode:  17046
Episode:  17047
Episode:  17048
Episode:  17049
Episode:  17050
Episode:  17051
Episode:  17052
Episode:  17053
Episode:  17054
Episode:  17055
Episode:  17056
Episode:  17057
Episode:  17058
Episode:  17059
Episode:  17060
Episode:  17061
Episode:  17062
Episode:  17063
Episode:  17064
Episode:  17065
Episode:  17066
Episode:  17067
Episode:  17068
Episode:  17069
Episode:  17070
Episode:  17071
Episode:  17072
Episode:  17073
Episode:  17074
Episode:  17075
Episode:  17076
Episode:  17077
Episode:  17078
Episode:  17079
Episode:  17080
Episode:

Episode:  19964
Episode:  19965
Episode:  19966
Episode:  19967
Episode:  19968
Episode:  19969
Episode:  19970
Episode:  19971
Episode:  19972
Episode:  19973
Episode:  19974
Episode:  19975
Episode:  19976
Episode:  19977
Episode:  19978
Episode:  19979
Episode:  19980
Episode:  19981
Episode:  19982
Episode:  19983
Episode:  19984
Episode:  19985
Episode:  19986
Episode:  19987
Episode:  19988
Episode:  19989
Episode:  19990
Episode:  19991
Episode:  19992
Episode:  19993
Episode:  19994
Episode:  19995
Episode:  19996
Episode:  19997
Episode:  19998
Episode:  19999
Episode: 20000 Reward: 8991.0 


In [45]:

#plt.plot(range(EPOCHS),np.cumsum(rewards), '\n')

np.set_printoptions(precision=4)
print('Q Table = \n', q_table)
print('Rewards for the last 10 episodes = ', rewards[-10:])


Q Table = 
 [[9.0893e-02 3.6131e-02 1.9371e-02 4.8541e-02]
 [1.4920e-04 6.1298e-03 4.1832e-03 1.1945e-01]
 [4.1021e-03 3.2825e-03 7.8939e-03 2.3066e-02]
 [4.5785e-05 5.3862e-03 2.3573e-03 1.3797e-02]
 [1.2832e-01 2.3275e-02 2.5293e-05 7.4351e-02]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.5482e-07 8.9921e-14 7.7561e-03 3.3473e-08]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.9668e-02 6.5393e-03 1.0408e-02 2.3611e-01]
 [2.0250e-02 3.9677e-01 2.9987e-03 1.8109e-02]
 [1.2973e-01 1.7415e-05 1.5302e-02 1.1715e-03]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [6.8163e-02 2.0462e-02 8.2972e-01 7.4060e-02]
 [1.2506e-01 9.9542e-01 1.9224e-01 1.1362e-01]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]]
Rewards for the last 10 episodes =  [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]


In [46]:
#After Training

In [47]:
env=gym.make("FrozenLake-v1", render_mode='human', is_slippery=slipperyState)
env.reset()
state = env.reset()[0]
time.sleep(1)
for steps in range(100):
    env.render()
    action = np.argmax(q_table[state, :])
    state, reward, done, done1, info = env.step(action)
    if done:
        break
env.close()

In [48]:
#plt.plot(range(EPOCHS),np.cumsum(rewards), '\n')

np.set_printoptions(precision=4)
print('Q Table = \n', q_table)
print('Rewards for the last 10 episodes = ', rewards[-10:])

Q Table = 
 [[9.0893e-02 3.6131e-02 1.9371e-02 4.8541e-02]
 [1.4920e-04 6.1298e-03 4.1832e-03 1.1945e-01]
 [4.1021e-03 3.2825e-03 7.8939e-03 2.3066e-02]
 [4.5785e-05 5.3862e-03 2.3573e-03 1.3797e-02]
 [1.2832e-01 2.3275e-02 2.5293e-05 7.4351e-02]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.5482e-07 8.9921e-14 7.7561e-03 3.3473e-08]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.9668e-02 6.5393e-03 1.0408e-02 2.3611e-01]
 [2.0250e-02 3.9677e-01 2.9987e-03 1.8109e-02]
 [1.2973e-01 1.7415e-05 1.5302e-02 1.1715e-03]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [6.8163e-02 2.0462e-02 8.2972e-01 7.4060e-02]
 [1.2506e-01 9.9542e-01 1.9224e-01 1.1362e-01]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]]
Rewards for the last 10 episodes =  [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]


In [16]:
#During Training (Shown - for video)

In [17]:
time.sleep(5)

In [18]:
# Set up a new environment
env=gym.make("FrozenLake-v1", is_slippery=slipperyState)
env.reset()
state = env.reset()[0]

#Check action size and atate size
action_size = env.action_space.n
state_size = env.observation_space.n   
print('action size=', action_size, 'state size=', state_size)

# Start with very small values for all our Q(s,a)
q_table = np.zeros([state_size, action_size])
print('Q Table Init = \n', q_table, '\nQ Table Shape = ', q_table.shape)

action size= 4 state size= 16
Q Table Init = 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 
Q Table Shape =  (16, 4)


In [19]:
# It is common to leave Hyperparameters in ALL CAPS to easily locate them

EPOCHS=20000  # number of epochs/episodes to train for
ALPHA = 0.8 # aka the learning rate
GAMMA = 0.95 # aka the discount rate
# MAX_EPISODES = 100  # optional, also defined in env setup above

# Exploration vs. Exploitation parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.001             # Exponential decay rate for exploration prob

def epsilon_greedy_action_selection(epsilon, q_table, discrete_state):
    '''
    Returns an action for the agent. Note how it uses a random number to decide on
    exploration versus explotation trade-off.
    '''
    random_number = np.random.random()
    
    # EXPLOITATION, USE BEST Q(s,a) Value
    if random_number > epsilon:
        # Action row for a particular state
        state_row = q_table[discrete_state,:]
        # Index of highest action for state
        # Recall action is mapped to index (e.g. 0=LEFT, 1=DOWN, etc..)
        action = np.argmax(state_row)
    
    # EXPLORATION, USE A RANDOM ACTION
    else:
        # Return a random 0,1,2,3 action
        action = env.action_space.sample()
        
    return action

def compute_next_q_value(old_q_value, reward, next_optimal_q_value):
    
    return old_q_value +  ALPHA * (reward + GAMMA * next_optimal_q_value - old_q_value)

def reduce_epsilon(epsilon,epoch):
    
    return min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*epoch)

In [20]:
# Reset just in case, watch lecture on this.
q_table = np.zeros([state_size, action_size])
total_reward = 0
epsilon = 1

# List of rewards
rewards = []

In [21]:
env=gym.make("FrozenLake-v1", render_mode='human', is_slippery=slipperyState)
env.reset()
state = env.reset()[0]

# Play 20k games
for episode in range(EPOCHS):
    print('Episode: ',episode)
    # Reset the environment
    state = env.reset()[0]
    done = False
    done1 = False
    total_rewards = 0
    
    while not done:
        action = epsilon_greedy_action_selection(epsilon,q_table, state)

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, done1, info = env.step(action)

        
        # Look up current/old qtable value Q(s_t,a_t)
        # print('state= ', state, 'action= ', action)
        old_q_value =  q_table[state,action]  

        # Get the next optimal Q-Value
        next_optimal_q_value = np.max(q_table[new_state, :])  

        # Compute next q value
        next_q = compute_next_q_value(old_q_value, reward, next_optimal_q_value)   

        # Update Q Table
        q_table[state,action] = next_q

        
        
        total_rewards = total_rewards + reward
        
        # Our new state is state
        state = new_state
                
    episode += 1
    # Reduce epsilon (because we need less and less exploration)
    epsilon = reduce_epsilon(epsilon,episode) 
    rewards.append(total_rewards)
    if episode % 1000 == 0:
        print('Episode: {} Reward: {} '.format(episode,np.sum(rewards))) 
            

env.close()

Episode:  0
Episode:  1
Episode:  2
Episode:  3
Episode:  4
Episode:  5
Episode:  6
Episode:  7
Episode:  8
Episode:  9
Episode:  10
Episode:  11
Episode:  12
Episode:  13
Episode:  14
Episode:  15
Episode:  16
Episode:  17
Episode:  18
Episode:  19
Episode:  20
Episode:  21
Episode:  22
Episode:  23
Episode:  24
Episode:  25
Episode:  26
Episode:  27
Episode:  28
Episode:  29
Episode:  30
Episode:  31
Episode:  32


KeyboardInterrupt: 

In [ ]:
#plt.plot(range(EPOCHS),np.cumsum(rewards), '\n')

np.set_printoptions(precision=4)
print('Q Table = \n', q_table)
print('Rewards for the last 10 episodes = ', rewards[-10:])